# Frost Days Calculation for CHLSA_V2 Monthly Data - Jupyter Notebook

This Jupyter Notebook contains a script for calculating frost days from CHLSA_V2 monthly temperature data for Switzerland. Frost days are defined as the days when the temperature falls below a certain threshold, typically -2°C. The processed data is saved with updated values representing the number of frost days for each month and year.

## Requirements

1. **ArcGIS Software**: The script uses arcpy, which is a part of the ArcGIS software suite. Ensure that you have a compatible version of ArcGIS installed.

2. **Input Data**: Prepare the input raster temperature data from the CHLSA_V2 dataset for Switzerland. The data should be located in the specified folder.

## Usage

1. **Open the Jupyter Notebook**: Launch your Jupyter Notebook environment.

2. **Run the Notebook Cells**: Execute the notebook cells sequentially by clicking on each cell and pressing Shift + Enter. Make sure to run the cells in the correct order.

3. **Output**: The notebook processes the input temperature data and calculates the number of frost days. Processed data files with updated values for frost days are saved in the specified output folder.

## Notebook Workflow

1. **Import Packages**: Required packages including arcpy, os, time, and re are imported. The "Spatial" extension for arcpy is checked out.

2. **Environment Setup**: Workspace and overwrite settings are configured to manage input and output paths.

3. **Folder Configuration**: The folder containing the input temperature data is specified in the `folders` list.

4. **Data Elaboration**:
   - The notebook iterates through each folder specified in the `folders` list.
   - Input and output paths are defined based on the specified folders.
   - The notebook extracts the list of .tif files in the input directory.
   - For each input file, the notebook performs the following steps:
     - Extracts the month and year from the file name using regular expressions.
     - Generates a new filename based on the month and year.
     - Reads the input raster as a NumPy array.
     - Applies a condition using NumPy operations to calculate frost days.
     - Saves the updated temperature raster to a new file with frost day values.

5. **Progress and Information**: Progress and relevant information for each processed file are displayed in the notebook's output cells.

## Notes

- Verify the correctness of regular expressions to extract month and year from file names.
- Double-check the condition and operations applied to calculate frost days.
- Review the script logic and arcpy functions to ensure they match your data processing requirements.
- It's recommended to review and verify the processed data to ensure accuracy.

## Author

Notebook written by Luca Ferrari

Contact: luca.ferrari@usys.ethz.ch

For inquiries or assistance, please contact the author.

This README content was generated with the assistance of an AI language model from OpenAI. The provided content is based on user input and has been tailored to the specific requirements of the project.

In [ ]:
# %% Import packages
import arcpy
from arcpy import env
import os
import time
import re

arcpy.CheckOutExtension("Spatial")

In [ ]:
env.workspace = r"N:\Luca_data"
arcpy.env.overwriteOutput = True

folders = ["Output_Switzerland_tasmin_celsius_LV95"]

In [ ]:
# %% Elaborate data
for folder in folders:
    # Specify the input path
    input_path = os.path.join(env.workspace, "Chelsa_V2_Monthly", folder)

    # Specify the output path
    output_path = os.path.join(env.workspace, "Chelsa_V2_Monthly\Output_Switzerland_fst_LV95")
    
    # Create output folder if it does not exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Get the list of files in the directory using os.scandir()
    with os.scandir(input_path) as entries:
        # Filter out directories and get only file names
        file_names = [entry.name for entry in entries if entry.is_file()]
        # Filter the list to only include .tif files
        file_names = [f for f in file_names if f.endswith('.tif')]

    # Process data for each file name and output file path
    for file_name in file_names:
        start = time.time()
        # Construct full paths to the file and output
        file_path = os.path.join(input_path, file_name)

        # Extract month and year using regular expressions
        match = re.search(r"_\d{2}_(\d{4})", file_name)
        month = match.group(0)[1:3]  # Extract month substring
        year = match.group(1)  # Extract year substring

        # Generate new filename
        new_filename = f"Output_Switzerland_fst_{month}_{year}.tif"
        output_file_path =  os.path.join(output_path, new_filename)

        # Read the input raster as a NumPy array
        input_raster = arcpy.Raster(file_path)

        # Apply the condition using NumPy operations
        frost_days = arcpy.sa.Con(input_raster < -2, 11.62 - input_raster * 1.57, 0.0)
        frost_days = arcpy.sa.Con(frost_days > 30, 30, frost_days)

        # Save the updated temperature raster to a new file
        frost_days.save(output_file_path)
    
        end = time.time()
        print('The file took', end - start, 's to be processed')

        # Print progress or other relevant information
        print(f"Processed: {file_path} -> {output_file_path}\n")
